In [1]:
import pandas as pd
import re
from time import time
from pprint import pprint

In [2]:
from pyspark.mllib.fpm import FPGrowth

In [3]:
#config
datafile = '/home/test/Documents/DataCleansing/Transaction Classification Problem/cut_dataset2.csv'
processed_data_out='/home/test/Documents/DataCleansing/Transaction Classification Problem/Processed2.csv'
fp_tree_1M = '/home/test/Documents/DataCleansing/Transaction Classification Problem/fp_tree2.csv'

parts = 10
num_obs=10

In [4]:
num_rep='\d{4,}'

In [5]:
2+4

6

In [6]:
data = sc.textFile(datafile,minPartitions=parts)

In [7]:
data.count()

38381240

In [8]:
# data = sc.textFile(datafile,minPartitions=parts)
# data = data.cache()

header = data.first()
data = data.filter(lambda line: line != header)
# print(data.take(num_obs),'\n')
data = data.map(lambda x: x.split("|"))
# print(data.take(num_obs),'\n')
data = data.map(lambda x: x[1])
# print(data.take(num_obs),'\n')
data = data.map(lambda line:re.sub('[^A-Za-z0-9]+',',', line))
print(data.take(num_obs),'\n')
data = data.map(lambda line:re.sub(num_rep, 'NUM', line))
print(data.take(num_obs),'\n')
data = data.map(lambda line: line.lower().strip().split(',')[:3])
print(data.take(num_obs),'\n')
data = data.map(lambda line: [elem+'_P'+str(idx+1) for (elem,idx) in zip(line,range(len(line)))])
print(data.take(num_obs),'\n')


['TRF,IFO,CRSG,SALARY,ACCOUNT', 'FEE,ON,OD,BOOKING', 'SHORTFALL,IRO,CRSG,TOD,TRANSACTION', 'PART,PYMT,2008,2014,PAYE,TAX,AUDIT', 'TD052211000031,Interest,adjustment', 'TD052211000031,II,Perd,2,14', 'Closure,proceeds,credited,to,TD052211000048,', '2022681137,Additional,amount,for,extension', 'TD007411000169,TDS', 'TD007411000169,Closure,Proceeds'] 

['TRF,IFO,CRSG,SALARY,ACCOUNT', 'FEE,ON,OD,BOOKING', 'SHORTFALL,IRO,CRSG,TOD,TRANSACTION', 'PART,PYMT,NUM,NUM,PAYE,TAX,AUDIT', 'TDNUM,Interest,adjustment', 'TDNUM,II,Perd,2,14', 'Closure,proceeds,credited,to,TDNUM,', 'NUM,Additional,amount,for,extension', 'TDNUM,TDS', 'TDNUM,Closure,Proceeds'] 

[['trf', 'ifo', 'crsg'], ['fee', 'on', 'od'], ['shortfall', 'iro', 'crsg'], ['part', 'pymt', 'num'], ['tdnum', 'interest', 'adjustment'], ['tdnum', 'ii', 'perd'], ['closure', 'proceeds', 'credited'], ['num', 'additional', 'amount'], ['tdnum', 'tds'], ['tdnum', 'closure', 'proceeds']] 

[['trf_P1', 'ifo_P2', 'crsg_P3'], ['fee_P1', 'on_P2', 'od_P3'], [

In [9]:
data.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 6.0 failed 1 times, most recent failure: Lost task 2.0 in stage 6.0 (TID 72, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/test/spark-2.2.0-bin-

In [10]:
data = data.sample(False, 0.1, seed=101)


In [30]:
data.saveAsTextFile(processed_data_out)

Py4JJavaError: An error occurred while calling o731.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 21.0 failed 1 times, most recent failure: Lost task 2.0 in stage 21.0 (TID 87, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 138, in dump_stream
    for obj in iterator:
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/pyspark/rdd.py", line 1541, in func
    for x in iterator:
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/pyspark/rddsampler.py", line 81, in func
    for obj in ite

In [11]:
datamodel = FPGrowth.train(data,minSupport=0.001,numPartitions=10)


Py4JJavaError: An error occurred while calling o235.trainFPGrowthModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 16.0 failed 1 times, most recent failure: Lost task 2.0 in stage 16.0 (TID 26, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/test/spark-2.2.0-bin-hadoop2.7/python/pyspark/rddsampler.py", line 81, in func
    for obj in iterator:
  File "<ipython-input-9-6c5caaaa13fa>", line 9, in <lambda>
IndexError: lis

In [14]:
df=data.toDF()
# data.take(100)
data.getNumPartitions()

10

In [15]:
df.head(4),df.first()

([Row(_1='trf_P1', _2='ifo_P2', _3='crsg_P3'),
  Row(_1='fee_P1', _2='on_P2', _3='od_P3'),
  Row(_1='shortfall_P1', _2='iro_P2', _3='crsg_P3'),
  Row(_1='part_P1', _2='pymt_P2', _3='num_P3')],
 Row(_1='trf_P1', _2='ifo_P2', _3='crsg_P3'))

In [17]:
result = datamodel.freqItemsets().collect()
print(len(result))
result[:10],result[-10:]

144


([FreqItemset(items=['num_P1'], freq=1644),
  FreqItemset(items=['mobpymt_P1'], freq=9246),
  FreqItemset(items=['mobpymt_P1', 'pp_P3'], freq=9238),
  FreqItemset(items=['mobpymt_P1', 'pp_P3', '4qtlnum_P2'], freq=9238),
  FreqItemset(items=['mobpymt_P1', '4qtlnum_P2'], freq=9246),
  FreqItemset(items=['cw_P1'], freq=2175),
  FreqItemset(items=['cw_P1', 'by_P2'], freq=1088),
  FreqItemset(items=['self_P2'], freq=16961),
  FreqItemset(items=['self_P2', 'cdb_P1'], freq=6157),
  FreqItemset(items=['self_P2', 'cdb_P1', 'num_P3'], freq=5684)],
 [FreqItemset(items=['v_P1'], freq=1149),
  FreqItemset(items=['by_P3'], freq=5660),
  FreqItemset(items=['by_P3', 'cd_P1'], freq=2821),
  FreqItemset(items=['vlt_P3'], freq=1660),
  FreqItemset(items=['vlt_P3', 'to_P2'], freq=1557),
  FreqItemset(items=['pp_P3'], freq=9756),
  FreqItemset(items=['pp_P3', '4qtlnum_P2'], freq=9756),
  FreqItemset(items=['by_P2'], freq=17954),
  FreqItemset(items=['vlt_P1'], freq=2297),
  FreqItemset(items=['vlt_P1', 'to

In [18]:
data_length = data.count()
data_length

999999

In [19]:
#creating list of fp_tree
fp_tree_list =[[item[0],item[1],round(100*item[1]/data_length,3)] for item in result]
len(fp_tree_list)
pprint(fp_tree_list[:5])

[[['num_P1'], 1644, 0.164],
 [['mobpymt_P1'], 9246, 0.925],
 [['mobpymt_P1', 'pp_P3'], 9238, 0.924],
 [['mobpymt_P1', 'pp_P3', '4qtlnum_P2'], 9238, 0.924],
 [['mobpymt_P1', '4qtlnum_P2'], 9246, 0.925]]


In [20]:
#create dictionary of Fp tree
fp_tree_dict = {}
i=0
for elem in result:
    i+=1
    fp_tree_dict[str(elem[0])]=[elem[1],round(100*elem[1]/data_length,2)]
# result[0][1]
# print(fp_tree_dict.items())

In [21]:
#creating dataframe of FP tree
import pandas as pd
fp_df = pd.DataFrame(fp_tree_list,columns=['tree_map', 'absolute_support', 'support_ratio'])
print(fp_df.head())

                          tree_map  absolute_support  support_ratio
0                         [num_P1]              1644          0.164
1                     [mobpymt_P1]              9246          0.925
2              [mobpymt_P1, pp_P3]              9238          0.924
3  [mobpymt_P1, pp_P3, 4qtlnum_P2]              9238          0.924
4         [mobpymt_P1, 4qtlnum_P2]              9246          0.925


In [22]:
fp_df['num_Ps']=fp_df['tree_map'].apply(lambda x:len(x))

In [23]:
fp_df.head(20)

,tree_map,absolute_support,support_ratio,num_Ps
0,[num_P1],1644,0.164,1
1,[mobpymt_P1],9246,0.925,1
2,"[mobpymt_P1, pp_P3]",9238,0.924,2
3,"[mobpymt_P1, pp_P3, 4qtlnum_P2]",9238,0.924,3
4,"[mobpymt_P1, 4qtlnum_P2]",9246,0.925,2
5,[cw_P1],2175,0.218,1
6,"[cw_P1, by_P2]",1088,0.109,2
7,[self_P2],16961,1.696,1
8,"[self_P2, cdb_P1]",6157,0.616,2
9,"[self_P2, cdb_P1, num_P3]",5684,0.568,3


In [24]:
fp_df.num_Ps.value_counts()

1    66
2    63
3    15
Name: num_Ps, dtype: int64

In [25]:
fp_df.groupby('num_Ps').agg('mean')

,absolute_support,support_ratio
num_Ps,,
1,37040.590909,3.704152
2,32883.587302,3.288444
3,40760.066667,4.076133


In [26]:
def create_elem(l,i):
    if len(l) > i:
        return l[i]
    else:
        return ''

In [27]:
fp_df['Key_1'] = fp_df['tree_map'].apply(lambda l:l[0])
fp_df['Key_2'] = fp_df['tree_map'].apply(create_elem,args=(1,))
fp_df['Key_3'] = fp_df['tree_map'].apply(create_elem,args=(2,))

print(len(fp_df[fp_df['Key_3'] != '']))
fp_df[fp_df['Key_3'] != '']

15


,tree_map,absolute_support,support_ratio,num_Ps,Key_1,Key_2,Key_3
3,"[mobpymt_P1, pp_P3, 4qtlnum_P2]",9238,0.924,3,mobpymt_P1,pp_P3,4qtlnum_P2
9,"[self_P2, cdb_P1, num_P3]",5684,0.568,3,self_P2,cdb_P1,num_P3
17,"[no_P2, chq_P1, num_P3]",1210,0.121,3,no_P2,chq_P1,num_P3
23,"[cwd_P1, self_P3, by_P2]",1395,0.140,3,cwd_P1,self_P3,by_P2
31,"[ifo_P3, num_P2, chq_P1]",3921,0.392,3,ifo_P3,num_P2,chq_P1
40,"[wd_P2, atm_P1, num_P3]",521516,52.152,3,wd_P2,atm_P1,num_P3
59,"[ngn_P3, revaluation_P1, of_P2]",3778,0.378,3,ngn_P3,revaluation_P1,of_P2
64,"[pos_P1, pur_P2, num_P3]",1276,0.128,3,pos_P1,pur_P2,num_P3
71,"[3qtl002_P3, web_P1, trf_P2]",3055,0.306,3,3qtl002_P3,web_P1,trf_P2
76,"[zar_P3, revaluation_P1, of_P2]",1292,0.129,3,zar_P3,revaluation_P1,of_P2


In [28]:

fp_df.to_csv(fp_tree_1M,sep = '|')

In [29]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
print(sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp))
print(sc.parallelize([]).aggregate((0, 0), seqOp, combOp))
# |      (0, 0)

(10, 4)
(0, 0)


In [30]:
rdd = sc.parallelize([-5, -4, -3, -2, -1, 1, 2, 3, 4], 10)
add = lambda x, y: x + y
print(rdd.treeReduce(add))
print(rdd.treeReduce(add),1)
print(rdd.treeReduce(add),2)
print(rdd.treeReduce(add),5)
print(rdd.treeReduce(add),10)

-5
-5 1
-5 2
-5 5
-5 10
